Installing the required libraries

In [1]:
!pip install pandas
!pip install sentence-transformers
!pip install setfit
!pip install scikit-learn
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

Importing the necessary libraries

In [2]:
import pandas as pd
import json
import os
from setfit import SetFitModel, SetFitTrainer
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import Dataset
from sklearn.metrics import classification_report
from collections import defaultdict

In [3]:
BASE_MODEL = "BAAI/bge-base-en-v1.5"
RANDOM_SEED = 42
OUTPUT_PATH = 'output'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data_dir = "/content/drive/MyDrive/multilabel_classification"
df = pd.read_csv(os.path.join(data_dir, "data.csv"))

In [6]:
df["text"] = df["id"] + " " + df["readme"]
df.drop(columns=["id", "readme"], axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
target_list = list([col for col in df.columns if col!="text"])
target_list

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Dependency management',
 'Support',
 'Testing',
 'Reporting',
 'Mobile CI',
 'Community',
 'Security',
 'Chat',
 'IDEs',
 'Monitoring',
 'Code review',
 'Code quality',
 'Publishing',
 'Project management',
 'API management',
 'Open Source management',
 'Mobile',
 'Code search',
 'GitHub Sponsors',
 'Continuous integration',
 'Deployment',
 'Container CI',
 'Localization',
 'Utilities',
 'Desktop tools',
 'AI Assisted',
 'Time tracking',
 'Code Scanning Ready',
 'Backup Utilities',
 'Learning']

In [8]:
def prepare_multilabel_label_col(row, targets):
    values = [row[target] for target in targets]

    return values


In [9]:
from sklearn.model_selection import train_test_split

df_copy = df.copy()
df_copy["label"] = df_copy.apply(lambda x: prepare_multilabel_label_col(x, target_list), axis=1)

df_copy.drop(columns=target_list, axis=1, inplace=True)

/usr/local/lib/python3.10/dist-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [10]:
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-mpnet-base-v2",
    multi_target_strategy="one-vs-rest",
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [11]:
def get_prediction_for_each_label(predictions, target_list):
    values = {}
    for i, target in enumerate(target_list):
        values["{0}_prediction".format(target)] = predictions[i]

    return values

In [12]:
def get_true_value_for_each_label(label, target_list):
    values = {}
    for i, target in enumerate(target_list):
        values["{0}_label".format(target)] = label[i]

    return values

In [13]:
def get_test_data_with_complete_values(test_df, y_pred, target_list):
    updated_test_data = []

    count = 0
    for i, row in test_df.iterrows():
        predictions = get_prediction_for_each_label(row["label"], target_list)
        row_labels = get_true_value_for_each_label(y_pred[count], target_list)

        for target in target_list:
            row["{0}_prediction".format(target)] = predictions["{0}_prediction".format(target)]
            row["{0}_label".format(target)] = row_labels["{0}_label".format(target)]

        updated_test_data.append(row)
        count += 1

    test_data = pd.DataFrame(updated_test_data)

    return test_data

In [14]:
import json
from sklearn.metrics import accuracy_score

def get_all_and_average_accuracy(test_data, target_list):
    accuracy_data = {}

    accuracy_scores = []
    for target in target_list:
        preds = test_data["{0}_prediction".format(target)].to_list()
        labels = test_data["{0}_label".format(target)].to_list()

        accuracy = accuracy_score(labels, preds)
        accuracy_data[target] = accuracy
        accuracy_scores.append(accuracy)


    accuracy_data = json.dumps(accuracy_data, indent = 4)
    average_accuracy_score = sum(accuracy_scores)/len(accuracy_scores)*100

    return accuracy_data, average_accuracy_score


In [15]:
def train_and_evaluate_model(train_set, test_set):
    trainer = SetFitTrainer(
        model=model,
        train_dataset=train_set,
        loss_class=CosineSimilarityLoss,
        metric="accuracy",
        batch_size=4,
        num_epochs=1,
        num_iterations=20,
    )
    trainer.train()
    y_pred = trainer.model.predict(test_set["text"])

    return y_pred

In [16]:
import numpy as np
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)

X = df_copy["text"]
y = df_copy["label"]

fold = 1
fold_accuracy_values = []
for train_index, test_index in kf.split(X, y):
    print(fold)
    train_df = df_copy.loc[train_index]
    test_df = df_copy.loc[test_index]

    train_set = Dataset.from_pandas(train_df)
    test_set = Dataset.from_pandas(test_df)

    y_pred = train_and_evaluate_model(train_set, test_set)
    updated_test_df = get_test_data_with_complete_values(test_df, y_pred, target_list)
    accuracy_data, average_accuracy = get_all_and_average_accuracy(updated_test_df, target_list)
    print("Accuracy data for Fold ", fold)
    print(json.dumps(accuracy_data, indent = 4))
    print("Average accuracy score for all labels: ", average_accuracy)
    fold_accuracy_values.append(average_accuracy)



1


<ipython-input-15-d6ceb4802b48>:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


Map:   0%|          | 0/970 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 38800
  Batch size = 4
  Num epochs = 1
  Total optimization steps = 9700


Step,Training Loss


...

Map:   0%|          | 0/971 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 38840
  Batch size = 4
  Num epochs = 1
  Total optimization steps = 9710


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 26 is present in all training examples.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/multiclass.py:77: UserWarning: Label not 28 is present in all training examples.
  warnings.warn(


Accuracy data for Fold  1
"{\n    \"Dependency management\": 0.9297520661157025,\n    \"Support\": 0.987603305785124,\n    \"Testing\": 0.9297520661157025,\n    \"Reporting\": 0.9586776859504132,\n    \"Mobile CI\": 1.0,\n    \"Community\": 0.9752066115702479,\n    \"Security\": 0.9669421487603306,\n    \"Chat\": 0.9793388429752066,\n    \"IDEs\": 1.0,\n    \"Monitoring\": 0.9834710743801653,\n    \"Code review\": 0.9380165289256198,\n    \"Code quality\": 0.9256198347107438,\n    \"Publishing\": 0.8760330578512396,\n    \"Project management\": 0.9256198347107438,\n    \"API management\": 0.9752066115702479,\n    \"Open Source management\": 0.9132231404958677,\n    \"Mobile\": 1.0,\n    \"Code search\": 0.9917355371900827,\n    \"GitHub Sponsors\": 1.0,\n    \"Continuous integration\": 0.9917355371900827,\n    \"Deployment\": 0.9462809917355371,\n    \"Container CI\": 0.9421487603305785,\n    \"Localization\": 1.0,\n    \"Utilities\": 0.9917355371900827,\n    \"Desktop tools\": 0.99586

In [17]:
average_cross_fold_accuracy = (sum(fold_accuracy_values)/ len(fold_accuracy_values))
print("Average cross-fold accuracy: ", average_cross_fold_accuracy)

Average cross-fold accuracy:  96.70320488839006
